In [6]:
import os 
import nltk
import math
import pandas as pd
import tabulate
import warnings
warnings.filterwarnings("ignore")
""" nltk.download('puntk')
nltk.download('stopwords') """
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer
from nltk.corpus import stopwords
from natsort import natsorted
stop_words = stopwords.words('english')
stop_words.remove('to')
stop_words.remove('in')
stop_words.remove('where')
document_of_terms = []
document_with_stemming = []
document_id = 1
positional_index = {}
ps = PorterStemmer()
file_name = natsorted(os.listdir('../DocumentCollection')) #to sort files in folder
for file in file_name :
    #1.1# Read 10 Documents
    with open(f'../DocumentCollection/{file}', 'r') as f:
        document = f.read()
    #1.2# apply tokenizing
    tokenizing = word_tokenize(document)
    for word in tokenizing : 
        if word not in stop_words :
            #1.3# apply stemming
            stemming = ps.stem(word)
        document_with_stemming.append(stemming)
    document_of_terms.append(document_with_stemming)
    document_with_stemming = []
#print(document_of_terms)
#{term : [freq, {docid : []}]}
#{antony : [10, {docid : [1,2,3]}]}
#2.1# positional index
for document in document_of_terms:
    for position, term in enumerate(document):
        if term in positional_index:
            positional_index[term][0] = positional_index[term][0] + 1
            if document_id in positional_index[term][1]:
                positional_index[term][1][document_id].append(position)
            else:
                positional_index[term][1][document_id] = [position]
        else:
            positional_index[term] = []
            positional_index[term].append(1)
            positional_index[term].append({})
            positional_index[term][1][document_id] = [position]
    document_id = document_id + 1
print(positional_index)
#2.5# phrase query
outer_array = [[] for i in range(10)]
query = "fools fear"
for wordd in query.split():
    word_after_stem = ps.stem(wordd)
    if word_after_stem in positional_index.keys():  
      for key in positional_index[word_after_stem][1].keys():
        if outer_array[key-1] != []:
            if outer_array[key-1][-1] == positional_index[word_after_stem][1][key][0] - 1:
                    outer_array[key-1].append(positional_index[word_after_stem][1][key][0])
        else:
                outer_array[key-1].append(positional_index[word_after_stem][1][key][0])
for position, arr in enumerate(outer_array, start=1):
       if len(arr) == len(query.split()):
            print(position)
#2.2# tf and weight tf
#tf
all_terms = []
for doc in document_of_terms:
    for word in doc:
        all_terms.append(word)
def term_frequency(doc):
    #put each term in dictionary form with value 0
    word_count = dict.fromkeys(all_terms, 0) 
    for word in doc:
        #increment the value of each term in dictionary
        word_count[word] += 1
        # return dictionary of each document
    return word_count
# convert dictionay form into columns form
tf = pd.DataFrame(term_frequency(document_of_terms[0]).values(), index = term_frequency(document_of_terms[0]). keys())
for i in range(1, len(document_of_terms)):
    tf[i] = term_frequency(document_of_terms[i]).values()
#change name of column
tf.columns = ['doc' + str(i) for i in range(1, 11)] 
#w_tf weighted term frequency 1 + log10(tf)
def w_tf(x):
    if x > 0:
        return math.log10(x) + 1
    else:
        return 0
for i in range(1, len(document_of_terms) + 1):
    #replace each raw with w_tf
    tf['doc'+ str(i)] =  tf['doc'+str(i)].apply(w_tf) 
#print(tf)
#2.3# IDF
df_and_IDF = pd.DataFrame(columns=('d_f','idf'))
for i in range(len(tf)):
    doc_freq = tf.iloc[i].values.sum()
    df_and_IDF.loc[i, 'd_f'] = doc_freq
    df_and_IDF.loc[i, 'idf'] = math.log10(10/float(doc_freq))
df_and_IDF.index = tf.index
#print(df_and_IDF)
#2.4# TF-IDF
tf_idf = tf.multiply(df_and_IDF['idf'], axis = 0)
#print(tf_idf)
#doc_length
import numpy as np
doc_length = pd.DataFrame()
def document_length(col):
    return np.sqrt(tf_idf[col].apply(lambda x: x**2).sum())
for column in tf_idf.columns:
    doc_length.loc[0, column + '_len'] = document_length(column)
#print(doc_length)
#NORMALIZATION TF-IDF divided by DOC_LENGTH
normalize = pd.DataFrame()
def get_normalize(col, x):
    try:
        return x / doc_length[column + '_len'].values[0]
    except:
        return 0
for column in tf_idf.columns:
    normalize[column] = tf_idf[column].apply(lambda x: get_normalize(column, x))




{'antoni': [3, {1: [0], 2: [0], 6: [0]}], 'brutu': [3, {1: [1], 2: [1], 4: [0]}], 'caeser': [5, {1: [2], 2: [2], 4: [1], 5: [0], 6: [1]}], 'cleopatra': [1, {1: [3]}], 'merci': [5, {1: [4], 3: [0], 4: [2], 5: [1], 6: [2]}], 'worser': [4, {1: [5], 3: [1], 4: [3], 5: [2]}], 'calpurnia': [1, {2: [3]}], 'angel': [3, {7: [0], 8: [0], 9: [0]}], 'fool': [4, {7: [1], 8: [1], 9: [1], 10: [0]}], 'fear': [3, {7: [2], 8: [2], 10: [1]}], 'in': [4, {7: [3], 8: [3], 9: [2], 10: [2]}], 'rush': [4, {7: [4], 8: [4], 9: [3], 10: [3]}], 'to': [4, {7: [5], 8: [5], 9: [4], 10: [4]}], 'tread': [4, {7: [6], 8: [6], 9: [5], 10: [5]}], 'where': [4, {7: [7], 8: [7], 9: [6], 10: [6]}]}
7
8
10


{'antoni': [3, {1: [0], 2: [0], 6: [0]}],
 'brutu': [3, {1: [1], 2: [1], 4: [0]}],
 'caeser': [5, {1: [2], 2: [2], 4: [1], 5: [0], 6: [1]}],
 'cleopatra': [1, {1: [3]}],
 'merci': [5, {1: [4], 3: [0], 4: [2], 5: [1], 6: [2]}],
 'worser': [4, {1: [5], 3: [1], 4: [3], 5: [2]}],
 'calpurnia': [1, {2: [3]}],
 'angel': [3, {7: [0], 8: [0], 9: [0]}],
 'fool': [4, {7: [1], 8: [1], 9: [1], 10: [0]}],
 'fear': [3, {7: [2], 8: [2], 10: [1]}],
 'in': [4, {7: [3], 8: [3], 9: [2], 10: [2]}],
 'rush': [4, {7: [4], 8: [4], 9: [3], 10: [3]}],
 'to': [4, {7: [5], 8: [5], 9: [4], 10: [4]}],
 'tread': [4, {7: [6], 8: [6], 9: [5], 10: [5]}],
 'where': [4, {7: [7], 8: [7], 9: [6], 10: [6]}]}

In [53]:
q = 'antony brutus'

In [54]:
import math
def wtf(x):
    if x > 0:
        return 1 + math.log(x)
    else:
        return 0

In [55]:
import pandas as pd
from numpy import count_nonzero
import math
query = pd.DataFrame(index = normalize.index)
x = []
for word in q.split():
    x.append(ps.stem(word))
#tf of query
query['tf'] = [x.count(term) if term in x else 0 for term in list(normalize.index)]
query['w_tf'] = query['tf'].apply(lambda x : int(wtf(x)))
query['idf'] = df_and_IDF['idf'] * query['w_tf']
query['tf-idf'] = query['tf'] * query['idf']
query['normalized'] = 0
product = normalize.multiply(query['w_tf'], axis = 0)
for i in range(len(query)):
    #normalizarion of each term in query
    query['normalized'].iloc[i] = float(query['idf'].iloc[i]) / math.sqrt(sum(query['idf'].values**2))
#normailze of each term in document * normalize of each term in query
product2 = product.multiply(query['normalized'], axis = 0)

print("Query Details")
query

Query Details


,tf,w_tf,idf,tf-idf,normalized
antoni,1,1,0.522879,0.522879,0.707107
brutu,1,1,0.522879,0.522879,0.707107
caeser,0,0,0.0,0.0,0.000000
cleopatra,0,0,0.0,0.0,0.000000
merci,0,0,0.0,0.0,0.000000
worser,0,0,0.0,0.0,0.000000
calpurnia,0,0,0.0,0.0,0.000000
angel,0,0,0.0,0.0,0.000000
fool,0,0,0.0,0.0,0.000000
fear,0,0,0.0,0.0,0.000000


In [28]:
scores = {}
#get product2 (normalize of document1*normalize of query, ormalize of document2*normalize of query)
for col in product2:
#if 0 is in any columns it is removed because no matter in sum 0
    if 0 in product2[col].loc[x].values:
        pass
    else:
        #sum of column1 that was normalize of doc1 * normalize of query
        scores[col] = product2[col].sum()


In [29]:
#query length
print("Query Length")
math.sqrt(sum([i**2 for i in query['idf'].loc[x]]))

Query Length


0.7394622130520805

In [48]:
#result of product normalize of document * normalize of query
result = product2[list(scores.keys())].loc[x]
print('Product (query*matched doc)')
print(result)

Product (query*matched doc)


,doc1,doc2
antoni,0.269196,0.288939
brutu,0.269196,0.288939


In [44]:
result.sum()

doc1    0.538393
doc2    0.577877
dtype: float64

In [49]:
final_scored = sorted(scores.items(), key = lambda x :x[1], reverse=True)
final_scored

[('doc2', 0.5778771030041435), ('doc1', 0.5383927937463102)]

In [41]:
for doc in final_scored:
    print(doc[0], end =' ')

doc2 doc1 